# HomeMatch

In [1]:
import os
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = str(os.environ.get("OPENAI_API_KEY"))
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
class RealEstateDescription(BaseModel):
    """Information of a property"""
    neighborhood: str = Field(..., description="Name of the neighborhood")
    price: int = Field(..., description="Listing price in USD")
    bedrooms: int = Field(..., description="Number of bedrooms")
    bathrooms: int = Field(..., description="Number of bathrooms")
    house_size: int = Field(..., description="Size of the house in square feet")
    description: str = Field(..., description="Detailed description of about 100 words of the property")
    neighborhood_description: str = Field(..., description="Description of the neighborhood")

class RealEstateListing(BaseModel):
    """A collection of real estate listings"""
    listing: List[RealEstateDescription] = Field(..., description="List of real estate property descriptions")


structured_llm = llm.with_structured_output(RealEstateListing)
prompt = "Generate a real estate listing of 20 properties"
listing = structured_llm.invoke(prompt)

/Users/dtabas/Library/CloudStorage/OneDrive-Danaher/Projects/Udacity/real_state_project/genai_exercise/.conda/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


RealEstateListing(listing=[RealEstateDescription(neighborhood='Downtown', price=500000, bedrooms=3, bathrooms=2, house_size=2000, description='Beautiful modern condo in the heart of downtown. Spacious living area with high ceilings and large windows. Updated kitchen with stainless steel appliances. Master bedroom with ensuite bathroom. Close to restaurants, shops, and public transportation.', neighborhood_description='Vibrant neighborhood with a mix of residential and commercial buildings.'), RealEstateDescription(neighborhood='Suburbia', price=400000, bedrooms=4, bathrooms=3, house_size=2500, description='Charming family home in a quiet suburban neighborhood. Open concept living and dining area with hardwood floors. Cozy fireplace in the living room. Large backyard with a deck for outdoor entertaining. Close to schools and parks.', neighborhood_description='Family-friendly community with tree-lined streets and parks.'), RealEstateDescription(neighborhood='Waterfront', price=700000, be